In [1]:
import wave
import numpy as np
from scipy.io import wavfile
from math import log10

In [2]:
def Encode():
    print("Audio Steganography ENCODING")
    print("")
    file = input("Enter audio file: ")
    song = wave.open(file, mode='rb')
    nframes = song.getnframes()
    frames = song.readframes(nframes)
    frame_list = list(frames)
    frame_byte = bytearray(frame_list)

    data = input("Enter the secret message:-")
    res = ''.join(format(i, '08b') for i in bytearray(data, encoding='utf-8'))
    data = data + '*^*^*'

    results = []
    for j in data:
        bits = bin(ord(j))[2:].zfill(8)
        results.extend([int(b) for b in bits])

    k = 0
    for i in range(0, len(results), 1):
        res = bin(frame_byte[k])[2:].zfill(8)
        if res[len(res) - 4] == results[i]:
            frame_byte[k] = (frame_byte[k] & 253)
        else:
            frame_byte[k] = (frame_byte[k] & 253) | 2
            frame_byte[k] = (frame_byte[k] & 254) | results[i]
        k = k + 1
    frame_modified = bytes(frame_byte)
    
    file_store = input("Enter audio file name to store encoded audio: ")
    with wave.open(file_store, 'wb') as fd:
        fd.setparams(song.getparams())
        fd.writeframes(frame_modified)

    song.close()
    print("Succesfully encoded inside audio file.")

In [3]:
def Decode():
    print("Audio Steganography DECODING")
    print("")
    file = input("Enter the audio file to be decoded: ")
    song = wave.open(file, mode='rb')
    nframes = song.getnframes()
    frames = song.readframes(nframes)
    frame_list = list(frames)
    frame_bytes = bytearray(frame_list)

    extracted = ""
    p = 0
    for i in range(len(frame_bytes)):
        if p == 1:
            break
        res = bin(frame_bytes[i])[2:].zfill(8)
        if res[len(res) - 2] == 0:
            extracted += res[len(res) - 4]
        else:
            extracted += res[len(res) - 1]

        all_bytes = [extracted[i: i + 8] for i in range(0, len(extracted), 8)]
        decode_data = ""
        for byte in all_bytes:
            decode_data += chr(int(byte, 2))
            if decode_data[-5:] == '*^*^*':
                print("The Encoded data was:", decode_data[:-5])
                p = 1
                break

In [4]:
def PSNR(original, hidden):
    hidden = np.resize(hidden, original.shape)
    mse = np.mean((original - hidden) ** 2)
    MAX = np.amax(original)
    psnr = 10 * log10(MAX ** 2 / mse)
    return psnr

In [ ]:
def steganography(): 
    while(1):
        n = int(input("Audio Steganography \n1. Encode the data \n2. Decode the data \n3. PSNR Calculate\nSelect the option: "))  
        if (n == 1):  
            print("\nEncoding...")  
            Encode()  
  
        elif (n == 2):  
            print("\nDecoding...")  
            Decode()
            
        elif (n==3):
            original = input("Enter original image (with extension): ")  
            hidden = input("Enter compressed image (with extension): ")  
            fs1, original = wavfile.read(original)
            fs2, hidden = wavfile.read(hidden)
            value = PSNR(original, hidden)
            print(f"PSNR value is {value} dB")      
        else:  
            print("Exiting....")
            break
  
steganography()  